# 강의 정보 크롤링

## 헤더 따기

In [8]:
import selenium 
from selenium import webdriver
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import pickle
from bs4 import BeautifulSoup as bs
import requests
import lxml.html
import re

In [9]:
browser = webdriver.Chrome() 

In [10]:
browser.get('https://everytime.kr/timetable/2020/1')

In [11]:
# 로그인
browser.find_element_by_name('userid').send_keys('ID')
browser.find_element_by_name('password').send_keys('Password')
browser.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()

In [12]:
header={
    'Host': 'everytime.kr',
    'Connection': 'keep-alive',
    'Content-Length': '45',
    'Accept':'*/*',
    'Origin': 'https://everytime.kr',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://everytime.kr/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cookie': 'etsid=s%3AZDdSNuhS5AUdNj8pvHkGVqWM7HwX5QFI.RPVTNCOyipi1mrRYeV%2B7BfJ8DQa5Kal7bw%2BDtsQ4yKg; _ga=GA1.2.1837060412.1581574157; _gid=GA1.2.453030296.1581574157; _gat_gtag_UA_22022140_4=1'
}

## 소속코드 수집

In [13]:
# 초기값 지정
year = [2014,2015,2016,2017,2018,2019,2020]
semester_code =[1,2]
name=[]
prof=[]
prof_name = []
types=[]
class_code=[]
score = []
category = []
time = []
weight = []
remarks = []
link = []
semester = []

## 강의 정보 수집

In [15]:
lecture = requests.post(
                'https://api.everytime.kr/find/timetable/subject/list',
                headers = header,
                data = 'categoryId=201066&campusId=62&year=2020&semester=1&limitNum=150&startNum=0')
lecture

<Response [200]>

In [16]:
# 강의 정보 얻기
category_codes={}
for y in tqdm_notebook(year):
    for s in tqdm_notebook(semester_code):
        if y == 2020 and s == 2:
            break
        # 크롤링 할 학기 지정
        url = 'https://api.everytime.kr/find/timetable/subject/filter/list'
        data = 'year={}&semester={}'.format(y,s)
        
        # 과목영역 코드북 생성
        code = requests.post(url,headers=header,data=data)
        parsed = bs(code.text,'xml')
        code_dict={}
        for i in parsed.findAll('category') :
            try:
                code_dict[i['id']] = [i['name'],i['parentId']]
            except:
                code_dict[i['id']] = [i['name'],'없음']
                
            
        category_codes['-'.join([str(y),str(s)])] = code_dict
        category_code = list(code_dict.keys())
        

        for i in category_code:
            temps = []
            start = 0
            limit = 50
            
            while True:
                    lecture = requests.post(
                    'https://api.everytime.kr/find/timetable/subject/list',
                    headers = header,
                    data ='categoryId={}&campusId=62&year={}&semester={}&limitNum={}&startNum={}'.format(i,y,s,limit,start))
                    lecture = bs(lecture.text,'xml')
                    sub = lecture.findAll('subject')
                    temps = temps + sub
                    start += 50

                    if len(sub) == 0:
                        break

            for temp in temps:
                    name.append(temp['name'])
                    prof.append([temp['professor']])
                    prof_name.append(temp['professor'])
                    types.append(temp['type'])
                    class_code.append(temp['code'])
                    score.append(temp['lectureRate'])
                    category.append(i)
                    time.append(temp['time'])
                    weight.append(temp['credit'])
                    remarks.append(temp['notice'])
                    link.append('https://everytime.kr/lecture/view/{}'.format(temp['lectureId']))
                    semester.append('-'.join([str(y),str(s)]))
                    
lecture_raw = [name,prof,prof_name,types,class_code,score,category,time,weight,remarks,link,semester]
df = pd.DataFrame(lecture_raw).T

df.drop(1,axis=1,inplace=True)
df.columns=['과목명','교수명','과목분류','과목코드','평점','소속코드','시간','학점','비고','링크','학기']
cols = ['과목코드','평점','소속코드','시간','학점','비고','링크','학기']

# 데이터 저장
df.to_pickle('lecture_raw.csv')
with open ('category_codes.pickle','wb') as f:
    pickle.dump(category_codes,f)